Load in the iris dataset:

In [1]:
from sklearn.datasets import load_iris

X, y = load_iris(as_frame=True, return_X_y=True)

In [3]:
X

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

We need to convert our pandas dataframes into tensorflow `Dataset` objects.

In [9]:
import tensorflow as tf

train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test = tf.data.Dataset.from_tensor_slices((X_test, y_test))

We are going to show the model the data 20 times apiece, since each time it sees the same example, it will be at a different point in the training process. We shuffle the data so that it doesn't see examples in the same order every time. The `batch` part of the code shows the examples to the model 32 at a time (which only affects the speed of training, not the outcome).

In [11]:
train = train.repeat(20).shuffle(1000).batch(32)

In [12]:
test = test.batch(1)

# Model set-up

We set up the model so that we have 2 hidden layers containing 10 nodes apiece. The output layer has 3 nodes because we have three species that we are trying to classify. By using `Dense`, we are specifying that each node is connected to every node in the next layer.

The `activation` argument specifies our activation function $\phi$. The ReLU function is

For the output node, we want the output data to be interpretable. In our case, we want to be able to treat the output as a probability that the observation belongs to that particular species.

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation=tf.nn.relu),   # hidden layer
    tf.keras.layers.Dense(10, activation=tf.nn.relu),   # hidden layer
    tf.keras.layers.Dense(3, activation=tf.nn.softmax)  # output layer
])

We need a loss function that can handle data with more than two classes. We are using sparse categorical cross-entropy. We will use accuracy to show how well the model is performing.

In [14]:
model.compile(
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

In [15]:
model.fit(
    train,
    validation_data=test,
    epochs=10,
)

Epoch 1/10
 1/70 [..............................] - ETA: 46s - loss: 1.2447 - accuracy: 0.2500

2024-03-15 11:08:17.737809: I external/local_xla/xla/service/service.cc:168] XLA service 0x7088aea4df70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-15 11:08:17.737830: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce MX550, Compute Capability 7.5
2024-03-15 11:08:17.741363: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-15 11:08:17.749016: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1710500897.775393   83361 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


70/70 [==============================] - 1s 4ms/step - loss: 1.0366 - accuracy: 0.4223 - val_loss: 0.9649 - val_accuracy: 0.6579
Epoch 2/10
70/70 [==============================] - 0s 4ms/step - loss: 0.9025 - accuracy: 0.6504 - val_loss: 0.8150 - val_accuracy: 0.6579
Epoch 3/10
70/70 [==============================] - 0s 3ms/step - loss: 0.7549 - accuracy: 0.6696 - val_loss: 0.6601 - val_accuracy: 0.6579
Epoch 4/10
70/70 [==============================] - 0s 3ms/step - loss: 0.6219 - accuracy: 0.7009 - val_loss: 0.5395 - val_accuracy: 0.6579
Epoch 5/10
70/70 [==============================] - 0s 4ms/step - loss: 0.5214 - accuracy: 0.8031 - val_loss: 0.4653 - val_accuracy: 0.6842
Epoch 6/10
70/70 [==============================] - 0s 4ms/step - loss: 0.4445 - accuracy: 0.8768 - val_loss: 0.3866 - val_accuracy: 0.9211
Epoch 7/10
70/70 [==============================] - 0s 3ms/step - loss: 0.3794 - accuracy: 0.9286 - val_loss: 0.3275 - val_accuracy: 0.9737
Epoch 8/10
70/70 [=============

In [16]:
predict_X = [
  [5.1, 3.3, 1.7, 0.5],
  [5.9, 3.0, 4.2, 1.5],
  [6.9, 3.1, 5.4, 2.1],
]

predictions = model.predict(predict_X)

1/1 [==============================] - 0s 69ms/step


In [17]:
predictions[0]

array([9.6228272e-01, 3.7702501e-02, 1.4804093e-05], dtype=float32)

In [18]:
predictions[0].argmax()

0

In [20]:
for pred_dict, expected in zip(predictions, ["setosa", "versicolor", "virginica"]):
    predicted_index = pred_dict.argmax()
    predicted = load_iris().target_names[predicted_index]
    probability = pred_dict.max()
    tick_cross = "✓" if predicted == expected else "✗"
    print(f"{tick_cross} Prediction is '{predicted}' ({100 * probability:.1f}%), expected '{expected}'")

✓ Prediction is 'setosa' (96.2%), expected 'setosa'
✓ Prediction is 'versicolor' (81.2%), expected 'versicolor'
✓ Prediction is 'virginica' (70.6%), expected 'virginica'


What happens if we decrease the number of hidden layers?

In [22]:
hidden1_model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation=tf.nn.relu),   # hidden layer
    tf.keras.layers.Dense(3, activation=tf.nn.softmax)  # output layer
])
hidden1_model.compile(
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
hidden1_model.fit(
    train,
    validation_data=test,
    epochs=10,
)

Epoch 1/10
70/70 [==============================] - 1s 4ms/step - loss: 1.1330 - accuracy: 0.4844 - val_loss: 1.0449 - val_accuracy: 0.3947
Epoch 2/10
70/70 [==============================] - 0s 2ms/step - loss: 0.9321 - accuracy: 0.4353 - val_loss: 0.8834 - val_accuracy: 0.4211
Epoch 3/10
70/70 [==============================] - 0s 3ms/step - loss: 0.7946 - accuracy: 0.5897 - val_loss: 0.7433 - val_accuracy: 0.7632
Epoch 4/10
70/70 [==============================] - 0s 3ms/step - loss: 0.6677 - accuracy: 0.8665 - val_loss: 0.6183 - val_accuracy: 0.8421
Epoch 5/10
70/70 [==============================] - 0s 2ms/step - loss: 0.5527 - accuracy: 0.9246 - val_loss: 0.5001 - val_accuracy: 0.9737
Epoch 6/10
70/70 [==============================] - 0s 2ms/step - loss: 0.4580 - accuracy: 0.9473 - val_loss: 0.4160 - val_accuracy: 0.9737
Epoch 7/10
70/70 [==============================] - 0s 2ms/step - loss: 0.3799 - accuracy: 0.9545 - val_loss: 0.3428 - val_accuracy: 0.9737
Epoch 8/10
70/70 [==

We actually reach `val_accuracy` of 97.37% after 5 epochs, which is faster than the model with 2 hidden layers. Let's try it with 0 hidden layers:

In [23]:
hidden0_model = tf.keras.Sequential([
    tf.keras.layers.Dense(3, activation=tf.nn.softmax)  # output layer
])
hidden0_model.compile(
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
hidden0_model.fit(
    train,
    validation_data=test,
    epochs=10,
)

Epoch 1/10
70/70 [==============================] - 0s 3ms/step - loss: 0.7249 - accuracy: 0.6210 - val_loss: 0.5925 - val_accuracy: 0.6842
Epoch 2/10
70/70 [==============================] - 0s 3ms/step - loss: 0.5996 - accuracy: 0.7384 - val_loss: 0.5339 - val_accuracy: 0.6842
Epoch 3/10
70/70 [==============================] - 0s 3ms/step - loss: 0.5629 - accuracy: 0.7607 - val_loss: 0.5051 - val_accuracy: 0.7105
Epoch 4/10
70/70 [==============================] - 0s 2ms/step - loss: 0.5388 - accuracy: 0.7728 - val_loss: 0.4827 - val_accuracy: 0.8158
Epoch 5/10
70/70 [==============================] - 0s 3ms/step - loss: 0.5179 - accuracy: 0.8451 - val_loss: 0.4702 - val_accuracy: 0.6842
Epoch 6/10
70/70 [==============================] - 0s 3ms/step - loss: 0.4989 - accuracy: 0.8228 - val_loss: 0.4488 - val_accuracy: 0.9737
Epoch 7/10
70/70 [==============================] - 0s 2ms/step - loss: 0.4812 - accuracy: 0.8763 - val_loss: 0.4342 - val_accuracy: 0.9737
Epoch 8/10
70/70 [==

It looks like we don't actually need any hidden layers to get high validation accuracy.

What happens if we decrease the number of nodes in the hidden layers?